In [2]:
import cv2
import numpy as np
from PIL import Image

def overlay_frames(video_path, color_priority=None, scale_factor=1, target_size=None):
    """
    Overlay frames with HSV ranges for Blue/Green + Gray included per frame.
    Returns the overlayed image as a numpy array.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(frame_count)

    positions = [0, 425]
    frames = []

    for pos in positions:
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
        success, frame = cap.read()
        if success:
            frames.append(frame)

    cap.release()

    color_ranges = {
        "blue":   [(40, 40, 15), (170, 255, 255)],
        "green":  [(20, 50, 25),  (80, 255, 255)],
        "yellow": [(20, 100, 100), (35, 255, 255)],
    }

    # Gray in BGR space (± tolerance)
    def gray_mask(frame, tol=30):
        lower = np.array([169 - tol, 169 - tol, 169 - tol], dtype=np.uint8)
        upper = np.array([169 + tol, 169 + tol, 169 + tol], dtype=np.uint8)
        return cv2.inRange(frame, lower, upper) > 0

    if color_priority is None:
        color_priority = ["green", "blue"]

    priority_map = {color: i for i, color in enumerate(color_priority)}

    # Start with blank canvas
    overlay = np.zeros_like(frames[0], dtype=np.uint8)
    priority_layer = np.full(frames[0].shape[:2], 9999, dtype=np.int32)

    for frame in frames:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Priority colors (Blue + Green)
        for color in ["blue", "green"]:
            if color not in priority_map:
                continue
            lower, upper = color_ranges[color]
            mask = cv2.inRange(hsv, np.array(lower), np.array(upper)) > 0
            update_mask = mask & (priority_map[color] < priority_layer)
            for c in range(3):
                overlay[:, :, c][update_mask] = frame[:, :, c][update_mask]
            priority_layer[update_mask] = priority_map[color]

        # Gray (BGR tolerance) — included for every frame
        if "gray" in priority_map:
            mask = gray_mask(frame)
            update_mask = mask & (priority_map["gray"] < priority_layer)
            for c in range(3):
                overlay[:, :, c][update_mask] = frame[:, :, c][update_mask]
            priority_layer[update_mask] = priority_map["gray"]

        # Default: fill remaining non-black pixels
        default_mask = (cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) > 0) & (priority_layer == 9999)
        for c in range(3):
            overlay[:, :, c][default_mask] = frame[:, :, c][default_mask]
        priority_layer[default_mask] = 1000

    # Upscale if requested
    if target_size:
        overlay = cv2.resize(overlay, target_size, interpolation=cv2.INTER_LANCZOS4)
    elif scale_factor != 1:
        h, w = overlay.shape[:2]
        overlay = cv2.resize(
            overlay,
            (w * scale_factor, h * scale_factor),
            interpolation=cv2.INTER_LANCZOS4
        )

    return overlay


def create_2x2_grid(video_paths, output_path, color_priority=None, scale_factor=1, spacing=10, bg_color=(255, 255, 255)):
    """
    Create a 2x2 grid from 4 videos.
    
    Parameters:
    - video_paths: List of 4 video file paths
    - output_path: Where to save the output image
    - color_priority: Color priority for overlay
    - scale_factor: Scale factor for each video
    - spacing: Pixels between images in grid
    - bg_color: Background color (BGR) for spacing
    """
    if len(video_paths) != 4:
        raise ValueError("Must provide exactly 4 video paths")
    
    # Process all 4 videos
    overlays = []
    for video_path in video_paths:
        print(f"Processing: {video_path}")
        overlay = overlay_frames(video_path, color_priority, scale_factor)
        overlays.append(overlay)
    
    # Get dimensions (assuming all videos have same size)
    h, w = overlays[0].shape[:2]
    
    # Create grid canvas
    grid_h = 2 * h + spacing
    grid_w = 2 * w + spacing
    grid = np.full((grid_h, grid_w, 3), bg_color, dtype=np.uint8)
    
    # Place images in 2x2 grid
    # Top-left
    grid[0:h, 0:w] = overlays[0]
    # Top-right
    grid[0:h, w+spacing:2*w+spacing] = overlays[1]
    # Bottom-left
    grid[h+spacing:2*h+spacing, 0:w] = overlays[2]
    # Bottom-right
    grid[h+spacing:2*h+spacing, w+spacing:2*w+spacing] = overlays[3]
    
    # Save
    grid_rgb = cv2.cvtColor(grid, cv2.COLOR_BGR2RGB)
    Image.fromarray(grid_rgb).save(output_path, quality=100, subsampling=0)
    print(f"Grid saved to {output_path}")


# Example usage
video_paths = [
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_success.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_fail.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_success.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_fail.mp4",
]
create_2x2_grid(
    video_paths,
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\overlay_grid.png",
    color_priority=["green", "blue", 'yellow'],
    scale_factor=1,
    spacing=20,  # White space between images
    bg_color=(255, 255, 255)  # White background
)


Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_success.mp4
750
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_fail.mp4
750
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_success.mp4
750
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_fail.mp4
750
Grid saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\overlay_grid.png


In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def overlay_frames(video_path, color_priority=None, scale_factor=1, target_size=None):
    """
    Overlay frames with HSV ranges for Blue/Green + Gray included per frame.
    Returns the overlayed image as a numpy array.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    positions = [0, 425]
    frames = []

    for pos in positions:
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
        success, frame = cap.read()
        if success:
            frames.append(frame)

    cap.release()

    color_ranges = {
        "blue":   [(40, 40, 15), (170, 255, 255)],
        "green":  [(20, 50, 25),  (80, 255, 255)],
        "yellow": [(20, 100, 100), (35, 255, 255)],
    }

    # Gray in BGR space (± tolerance)
    def gray_mask(frame, tol=30):
        lower = np.array([169 - tol, 169 - tol, 169 - tol], dtype=np.uint8)
        upper = np.array([169 + tol, 169 + tol, 169 + tol], dtype=np.uint8)
        return cv2.inRange(frame, lower, upper) > 0

    if color_priority is None:
        color_priority = ["green", "blue"]

    priority_map = {color: i for i, color in enumerate(color_priority)}

    # Start with blank canvas
    overlay = np.zeros_like(frames[0], dtype=np.uint8)
    priority_layer = np.full(frames[0].shape[:2], 9999, dtype=np.int32)

    for frame in frames:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Priority colors (Blue + Green)
        for color in ["blue", "green"]:
            if color not in priority_map:
                continue
            lower, upper = color_ranges[color]
            mask = cv2.inRange(hsv, np.array(lower), np.array(upper)) > 0
            update_mask = mask & (priority_map[color] < priority_layer)
            for c in range(3):
                overlay[:, :, c][update_mask] = frame[:, :, c][update_mask]
            priority_layer[update_mask] = priority_map[color]

        # Gray (BGR tolerance) — included for every frame
        if "gray" in priority_map:
            mask = gray_mask(frame)
            update_mask = mask & (priority_map["gray"] < priority_layer)
            for c in range(3):
                overlay[:, :, c][update_mask] = frame[:, :, c][update_mask]
            priority_layer[update_mask] = priority_map["gray"]

        # Default: fill remaining non-black pixels
        default_mask = (cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) > 0) & (priority_layer == 9999)
        for c in range(3):
            overlay[:, :, c][default_mask] = frame[:, :, c][default_mask]
        priority_layer[default_mask] = 1000

    # Upscale if requested
    if target_size:
        overlay = cv2.resize(overlay, target_size, interpolation=cv2.INTER_LANCZOS4)
    elif scale_factor != 1:
        h, w = overlay.shape[:2]
        overlay = cv2.resize(
            overlay,
            (w * scale_factor, h * scale_factor),
            interpolation=cv2.INTER_LANCZOS4
        )

    return overlay

def create_2x2_grid(video_paths, output_path, column_titles=None, row_titles=None, 
                    color_priority=None, scale_factor=1, spacing=10, 
                    bg_color=(255, 255, 255), title_margin_top=80, title_margin_left=120, font_size=40):
    """
    Create a 2x2 grid from 4 videos with row and column titles.
    
    Parameters:
    - video_paths: List of 4 video file paths
    - output_path: Where to save the output image
    - column_titles: List of 2 titles for columns (default: ["Column 1", "Column 2"])
    - row_titles: List of 2 titles for rows (default: ["Row 1", "Row 2"])
    - color_priority: Color priority for overlay
    - scale_factor: Scale factor for each video
    - spacing: Pixels between images in grid
    - bg_color: Background color (RGB) for spacing
    - title_margin_top: Space for column titles at top (pixels)
    - title_margin_left: Space for row titles on left (pixels)
    - font_size: Font size for titles
    """
    if len(video_paths) != 4:
        raise ValueError("Must provide exactly 4 video paths")
    
    if column_titles is None:
        column_titles = ["Column 1", "Column 2"]
    if row_titles is None:
        row_titles = ["Row 1", "Row 2"]
    
    # Process all 4 videos
    overlays = []
    for video_path in video_paths:
        print(f"Processing: {video_path}")
        overlay = overlay_frames(video_path, color_priority, scale_factor)
        overlays.append(overlay)
    
    # Get dimensions (assuming all videos have same size)
    h, w = overlays[0].shape[:2]
    
    # Create grid canvas with margins for titles
    grid_h = 2 * h + spacing + title_margin_top
    grid_w = 2 * w + spacing + title_margin_left
    
    # Convert bg_color from RGB to BGR for OpenCV
    bg_color_bgr = bg_color[::-1] if len(bg_color) == 3 else bg_color
    grid = np.full((grid_h, grid_w, 3), bg_color_bgr, dtype=np.uint8)
    
    # Place images in 2x2 grid (with offsets for titles)
    x_offset = title_margin_left
    y_offset = title_margin_top
    
    # Top-left
    grid[y_offset:y_offset+h, x_offset:x_offset+w] = overlays[0]
    # Top-right
    grid[y_offset:y_offset+h, x_offset+w+spacing:x_offset+2*w+spacing] = overlays[1]
    # Bottom-left
    grid[y_offset+h+spacing:y_offset+2*h+spacing, x_offset:x_offset+w] = overlays[2]
    # Bottom-right
    grid[y_offset+h+spacing:y_offset+2*h+spacing, x_offset+w+spacing:x_offset+2*w+spacing] = overlays[3]
    
    # Convert to PIL for text drawing
    grid_rgb = cv2.cvtColor(grid, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(grid_rgb)
    draw = ImageDraw.Draw(img)
    
    # Try to load a nice font, fall back to default if unavailable
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
        font_bold = ImageFont.truetype("arialbd.ttf", font_size)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", font_size)
            font_bold = font
        except:
            font = ImageFont.load_default()
            font_bold = font
    
    # Draw column titles (centered above each column)
    for col, title in enumerate(column_titles):
        x_center = x_offset + w // 2 + col * (w + spacing)
        y_pos = title_margin_top // 3
        
        # Get text bounding box for centering
        bbox = draw.textbbox((0, 0), title, font=font_bold)
        text_width = bbox[2] - bbox[0]
        
        draw.text((x_center - text_width // 2, y_pos), title, 
                 fill=(0, 0, 0), font=font_bold)
    
    # Draw row titles (rotated 90 degrees, centered on left)
    for row, title in enumerate(row_titles):
        y_center = y_offset + h // 2 + row * (h + spacing)
        x_pos = title_margin_left // 6  # More spacing from edge
        
        # Create rotated text
        bbox = draw.textbbox((0, 0), title, font=font_bold)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        
        # Create a temporary image for the rotated text
        txt_img = Image.new('RGB', (text_width + 10, text_height + 10), bg_color)
        txt_draw = ImageDraw.Draw(txt_img)
        txt_draw.text((5, 5), title, fill=(0, 0, 0), font=font_bold)
        
        # Rotate and paste
        rotated = txt_img.rotate(90, expand=True)
        paste_y = y_center - rotated.height // 2
        img.paste(rotated, (x_pos, paste_y))
    
    # Save
    img.save(output_path, quality=100, subsampling=0)
    print(f"Grid with titles saved to {output_path}")


# Example usage
video_paths = [
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_success.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_fail.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_success.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_fail.mp4",
]

create_2x2_grid(
    video_paths,
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\overlay_grid.png",
    column_titles=["Near Miss", "Failure"],
    row_titles=["Scalar Attacker", "Vector Attacker"],
    color_priority=["green", "blue", 'yellow'],
    scale_factor=1,
    spacing=20,
    bg_color=(255, 255, 255),
    title_margin_top=80,    # Space for column titles
    title_margin_left=150,  # Increased left margin (was 100)
    font_size=40
)
# Example usage
video_paths = [
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_success.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_fail.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_success.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_fail.mp4",
]

create_2x2_grid(
    video_paths,
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\overlay_grid.png",
    column_titles=["Near Miss", "Failure"],
    row_titles=["Scalar Attacker", "Vector Attacker"],
    color_priority=["green", "blue", 'yellow'],
    scale_factor=1,
    spacing=20,
    bg_color=(255, 255, 255),  # White background
    title_margin_top=80,    # Space for column titles
    title_margin_left=150,  # Increased left margin (was 100)
    font_size=40  # Title font size
)

Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_success.mp4
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_fail.mp4
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_success.mp4
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_fail.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\overlay_grid.png
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_success.mp4
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_fail.mp4
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_success.mp4
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_fail.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\

In [36]:
def create_2x2_grid(video_paths, output_path, column_titles=None, row_titles=None, 
                    color_priority=None, scale_factor=1, spacing=10, 
                    bg_color=(255, 255, 255), title_margin_top=80, title_margin_left=120, 
                    font_size=40, add_scene_numbers=True, scene_number_positions=None):
    """
    Create a 2x2 grid from 4 videos with row and column titles and scene numbers.
    
    Parameters:
    - video_paths: List of 4 video file paths
    - output_path: Where to save the output image
    - column_titles: List of 2 titles for columns
    - row_titles: List of 2 titles for rows
    - color_priority: Color priority for overlay
    - scale_factor: Scale factor for each video
    - spacing: Pixels between images in grid
    - bg_color: Background color (RGB) for spacing
    - title_margin_top: Space for column titles at top (pixels)
    - title_margin_left: Space for row titles on left (pixels)
    - font_size: Font size for titles
    - add_scene_numbers: Whether to add scene numbers (1a-4b)
    - scene_number_positions: List of 8 (x, y) tuples for absolute positions of each number.
                              Order: [1a, 1b, 2a, 2b, 3a, 3b, 4a, 4b]
    """
    if len(video_paths) != 4:
        raise ValueError("Must provide exactly 4 video paths")
    
    if column_titles is None:
        column_titles = ["Column 1", "Column 2"]
    if row_titles is None:
        row_titles = ["Row 1", "Row 2"]
    
    # Default scene number positions (8 total)
    if scene_number_positions is None:
        scene_number_positions = [
            (200, 150), (200, 400),  # 1a, 1b
            (800, 150), (800, 400),  # 2a, 2b
            (200, 750), (200, 1000), # 3a, 3b
            (800, 750), (800, 1000)  # 4a, 4b
        ]
    
    # Process all 4 videos
    overlays = []
    for video_path in video_paths:
        print(f"Processing: {video_path}")
        overlay = overlay_frames(video_path, color_priority, scale_factor)
        overlays.append(overlay)
    
    # Get dimensions (assuming all videos have same size)
    h, w = overlays[0].shape[:2]
    
    # Create grid canvas with margins for titles
    grid_h = 2 * h + spacing + title_margin_top
    grid_w = 2 * w + spacing + title_margin_left
    
    # Convert bg_color from RGB to BGR for OpenCV
    bg_color_bgr = bg_color[::-1] if len(bg_color) == 3 else bg_color
    grid = np.full((grid_h, grid_w, 3), bg_color_bgr, dtype=np.uint8)
    
    # Place images in 2x2 grid (with offsets for titles)
    x_offset = title_margin_left
    y_offset = title_margin_top
    
    # Top-left
    grid[y_offset:y_offset+h, x_offset:x_offset+w] = overlays[0]
    # Top-right
    grid[y_offset:y_offset+h, x_offset+w+spacing:x_offset+2*w+spacing] = overlays[1]
    # Bottom-left
    grid[y_offset+h+spacing:y_offset+2*h+spacing, x_offset:x_offset+w] = overlays[2]
    # Bottom-right
    grid[y_offset+h+spacing:y_offset+2*h+spacing, x_offset+w+spacing:x_offset+2*w+spacing] = overlays[3]
    
    # Convert to PIL for text drawing
    grid_rgb = cv2.cvtColor(grid, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(grid_rgb)
    draw = ImageDraw.Draw(img)
    
    # Try to load a nice font, fall back to default if unavailable
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
        font_bold = ImageFont.truetype("arialbd.ttf", font_size)
        scene_font = ImageFont.truetype("arialbd.ttf", int(font_size * 0.7))
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", font_size)
            font_bold = font
            scene_font = font
        except:
            font = ImageFont.load_default()
            font_bold = font
            scene_font = font
    
    # Draw column titles (centered above each column)
    for col, title in enumerate(column_titles):
        x_center = x_offset + w // 2 + col * (w + spacing)
        y_pos = title_margin_top // 3
        
        # Get text bounding box for centering
        bbox = draw.textbbox((0, 0), title, font=font_bold)
        text_width = bbox[2] - bbox[0]
        
        draw.text((x_center - text_width // 2, y_pos), title, 
                 fill=(0, 0, 0), font=font_bold)
    
    # Draw row titles (rotated 90 degrees, centered on left)
    for row, title in enumerate(row_titles):
        y_center = y_offset + h // 2 + row * (h + spacing)
        x_pos = title_margin_left // 6
        
        # Create rotated text
        bbox = draw.textbbox((0, 0), title, font=font_bold)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        
        # Create a temporary image for the rotated text with padding
        padding = 20
        txt_img = Image.new('RGB', (text_width + padding * 2, text_height + padding * 2), bg_color)
        txt_draw = ImageDraw.Draw(txt_img)
        txt_draw.text((padding, padding), title, fill=(0, 0, 0), font=font_bold)
        
        # Rotate and paste
        rotated = txt_img.rotate(90, expand=True)
        paste_y = y_center - rotated.height // 2
        paste_x = x_pos
        img.paste(rotated, (paste_x, paste_y))
    
    # Add scene numbers (8 total: 1a, 1b, 2a, 2b, 3a, 3b, 4a, 4b)
    if add_scene_numbers:
        labels = ['1a', '1b', '2a', '2b', '3a', '3b', '4a', '4b']
        
        for idx, (num_x, num_y) in enumerate(scene_number_positions):
            scene_num = labels[idx]
            
            # Draw scene number with white background circle
            bbox = draw.textbbox((0, 0), scene_num, font=scene_font)
            text_width = bbox[2] - bbox[0]
            text_height = bbox[3] - bbox[1]
            
            # Draw white circle background
            circle_radius = max(text_width, text_height) // 2 + 12
            circle_center_x = num_x + text_width // 2
            circle_center_y = num_y + text_height // 2
            
            draw.ellipse([circle_center_x - circle_radius, 
                         circle_center_y - circle_radius,
                         circle_center_x + circle_radius, 
                         circle_center_y + circle_radius],
                        fill=(255, 255, 255), outline=(0, 0, 0), width=2)
            
            # Draw number
            draw.text((num_x, num_y), scene_num, fill=(0, 0, 0), font=scene_font)
    
    # Save
    img.save(output_path, quality=100, subsampling=0)
    print(f"Grid with titles saved to {output_path}")


# Example usage
video_paths = [
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_success.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_fail.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_success.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_fail.mp4",
]

create_2x2_grid(
    video_paths,
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\overlay_grid.png",
    column_titles=["Near Miss", "Failure"],
    row_titles=["Scalar Attacker", "Vector Attacker"],
    color_priority=["green", "blue", 'yellow'],
    scale_factor=1,
    spacing=20,
    bg_color=(255, 255, 255),
    title_margin_top=80,
    title_margin_left=80,
    font_size=40,
    add_scene_numbers=True,
    # Direct (x, y) positions for all 8 numbers
    scene_number_positions=[
        (610, 400),   # 1a (bottom right of top-left image)
        (180, 240),   # 1b (top left of top-left image)
        (1525, 400),  # 2a (bottom right of top-right image)
        (1110, 240),   # 2b (top left of top-right image)

        (600, 840),   # 3a (bottom right of bottom-left image)
        (180, 725),   # 3b (top left of bottom-left image)
        (1525, 840),  # 4a (bottom right of bottom-right image)
        (1110, 680)    # 4b (top left of bottom-right image)
    ]
)

Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_success.mp4
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_fail.mp4
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_success.mp4
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_fail.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\overlay_grid.png


## Spawn Graphic


In [38]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def overlay_frames(video_path, color_priority=None, scale_factor=1, target_size=None):
    """
    Overlay frames with HSV ranges for Blue/Green + Gray included per frame.
    Returns the overlayed image as a numpy array.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    positions = [0]
    frames = []

    for pos in positions:
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
        success, frame = cap.read()
        if success:
            frames.append(frame)

    cap.release()

    color_ranges = {
        "blue":   [(40, 40, 15), (170, 255, 255)],
        "green":  [(20, 50, 25),  (80, 255, 255)],
        "yellow": [(20, 100, 100), (35, 255, 255)],
    }

    # Gray in BGR space (± tolerance)
    def gray_mask(frame, tol=30):
        lower = np.array([169 - tol, 169 - tol, 169 - tol], dtype=np.uint8)
        upper = np.array([169 + tol, 169 + tol, 169 + tol], dtype=np.uint8)
        return cv2.inRange(frame, lower, upper) > 0

    if color_priority is None:
        color_priority = ["green", "blue"]

    priority_map = {color: i for i, color in enumerate(color_priority)}

    # Start with blank canvas
    overlay = np.zeros_like(frames[0], dtype=np.uint8)
    priority_layer = np.full(frames[0].shape[:2], 9999, dtype=np.int32)

    for frame in frames:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Priority colors (Blue + Green)
        for color in ["blue", "green"]:
            if color not in priority_map:
                continue
            lower, upper = color_ranges[color]
            mask = cv2.inRange(hsv, np.array(lower), np.array(upper)) > 0
            update_mask = mask & (priority_map[color] < priority_layer)
            for c in range(3):
                overlay[:, :, c][update_mask] = frame[:, :, c][update_mask]
            priority_layer[update_mask] = priority_map[color]

        # Gray (BGR tolerance) — included for every frame
        if "gray" in priority_map:
            mask = gray_mask(frame)
            update_mask = mask & (priority_map["gray"] < priority_layer)
            for c in range(3):
                overlay[:, :, c][update_mask] = frame[:, :, c][update_mask]
            priority_layer[update_mask] = priority_map["gray"]

        # Default: fill remaining non-black pixels
        default_mask = (cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) > 0) & (priority_layer == 9999)
        for c in range(3):
            overlay[:, :, c][default_mask] = frame[:, :, c][default_mask]
        priority_layer[default_mask] = 1000

    # Upscale if requested
    if target_size:
        overlay = cv2.resize(overlay, target_size, interpolation=cv2.INTER_LANCZOS4)
    elif scale_factor != 1:
        h, w = overlay.shape[:2]
        overlay = cv2.resize(
            overlay,
            (w * scale_factor, h * scale_factor),
            interpolation=cv2.INTER_LANCZOS4
        )

    return overlay


# Example usage
video_paths = [
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\close_tail.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\far_tail.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\outside_edge.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\inside_edge.mp4",
]

output_paths = [
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\close_tail.png",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\far_tail.png",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\outside_edge.png",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\inside_edge.png",
]

for i, video_path in enumerate(video_paths):
    print(f"Processing: {video_path}")
    overlay = overlay_frames(video_path, color_priority=None, scale_factor=1)

    overlay_rgb = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(overlay_rgb)

    # Save
    img.save(output_paths[i], quality=100, subsampling=0)
    print(f"Grid with titles saved to {output_paths[i]}")

Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\close_tail.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\close_tail.png
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\far_tail.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\far_tail.png
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\outside_edge.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\outside_edge.png
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\inside_edge.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\spawns\inside_edge.png


In [2]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def overlay_frames(video_path, color_priority=None, scale_factor=1, target_size=None):
    """
    Overlay frames with HSV ranges for Blue/Green + Gray included per frame.
    Returns the overlayed image as a numpy array.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    positions = [425]
    frames = []

    for pos in positions:
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
        success, frame = cap.read()
        if success:
            frames.append(frame)

    cap.release()

    color_ranges = {
        "blue":   [(40, 40, 15), (170, 255, 255)],
        "green":  [(20, 50, 25),  (80, 255, 255)],
        "yellow": [(20, 100, 100), (35, 255, 255)],
    }

    # Gray in BGR space (± tolerance)
    def gray_mask(frame, tol=30):
        lower = np.array([169 - tol, 169 - tol, 169 - tol], dtype=np.uint8)
        upper = np.array([169 + tol, 169 + tol, 169 + tol], dtype=np.uint8)
        return cv2.inRange(frame, lower, upper) > 0

    if color_priority is None:
        color_priority = ["green", "blue"]

    priority_map = {color: i for i, color in enumerate(color_priority)}

    # Start with blank canvas
    overlay = np.zeros_like(frames[0], dtype=np.uint8)
    priority_layer = np.full(frames[0].shape[:2], 9999, dtype=np.int32)

    for frame in frames:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Priority colors (Blue + Green)
        for color in ["blue", "green"]:
            if color not in priority_map:
                continue
            lower, upper = color_ranges[color]
            mask = cv2.inRange(hsv, np.array(lower), np.array(upper)) > 0
            update_mask = mask & (priority_map[color] < priority_layer)
            for c in range(3):
                overlay[:, :, c][update_mask] = frame[:, :, c][update_mask]
            priority_layer[update_mask] = priority_map[color]

        # Gray (BGR tolerance) — included for every frame
        if "gray" in priority_map:
            mask = gray_mask(frame)
            update_mask = mask & (priority_map["gray"] < priority_layer)
            for c in range(3):
                overlay[:, :, c][update_mask] = frame[:, :, c][update_mask]
            priority_layer[update_mask] = priority_map["gray"]

        # Default: fill remaining non-black pixels
        default_mask = (cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) > 0) & (priority_layer == 9999)
        for c in range(3):
            overlay[:, :, c][default_mask] = frame[:, :, c][default_mask]
        priority_layer[default_mask] = 1000

    # Upscale if requested
    if target_size:
        overlay = cv2.resize(overlay, target_size, interpolation=cv2.INTER_LANCZOS4)
    elif scale_factor != 1:
        h, w = overlay.shape[:2]
        overlay = cv2.resize(
            overlay,
            (w * scale_factor, h * scale_factor),
            interpolation=cv2.INTER_LANCZOS4
        )

    return overlay


# Example usage
video_paths = [
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_fail.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_fail.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_success.mp4",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_success.mp4",
]

output_paths = [
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\scalar_freeze.png",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\vector_freeze.png",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\scalar_freeze2.png",
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\vector_freeze2.png",
]

for i, video_path in enumerate(video_paths):
    print(f"Processing: {video_path}")
    overlay = overlay_frames(video_path, color_priority=None, scale_factor=1)

    overlay_rgb = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(overlay_rgb)

    # Save
    img.save(output_paths[i], quality=100, subsampling=0)
    print(f"Grid with titles saved to {output_paths[i]}")

Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_fail.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\scalar_freeze.png
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_fail.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\vector_freeze.png
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_scalar_success.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\scalar_freeze2.png
Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\collision_vector_success.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\vector_freeze2.png


In [7]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def overlay_frames(video_path, color_priority=None, scale_factor=1, target_size=None):
    """
    Overlay frames with HSV ranges for Blue/Green + Gray included per frame.
    Returns the overlayed image as a numpy array.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    positions = [0]
    positions = [0, 375, frame_count - 1]
    frames = []

    for pos in positions:
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos)
        success, frame = cap.read()
        if success:
            frames.append(frame)

    cap.release()

    color_ranges = {
        "blue":   [(40, 40, 15), (170, 255, 255)],
        "green":  [(20, 50, 25),  (80, 255, 255)],
        "yellow": [(20, 100, 100), (35, 255, 255)],
    }

    # Gray in BGR space (± tolerance)
    def gray_mask(frame, tol=30):
        lower = np.array([169 - tol, 169 - tol, 169 - tol], dtype=np.uint8)
        upper = np.array([169 + tol, 169 + tol, 169 + tol], dtype=np.uint8)
        return cv2.inRange(frame, lower, upper) > 0

    if color_priority is None:
        color_priority = ["green", "blue"]

    priority_map = {color: i for i, color in enumerate(color_priority)}

    # Start with blank canvas
    overlay = np.zeros_like(frames[0], dtype=np.uint8)
    priority_layer = np.full(frames[0].shape[:2], 9999, dtype=np.int32)

    for frame in frames:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Priority colors (Blue + Green)
        for color in ["blue", "green"]:
            if color not in priority_map:
                continue
            lower, upper = color_ranges[color]
            mask = cv2.inRange(hsv, np.array(lower), np.array(upper)) > 0
            update_mask = mask & (priority_map[color] < priority_layer)
            for c in range(3):
                overlay[:, :, c][update_mask] = frame[:, :, c][update_mask]
            priority_layer[update_mask] = priority_map[color]

        # Gray (BGR tolerance) — included for every frame
        if "gray" in priority_map:
            mask = gray_mask(frame)
            update_mask = mask & (priority_map["gray"] < priority_layer)
            for c in range(3):
                overlay[:, :, c][update_mask] = frame[:, :, c][update_mask]
            priority_layer[update_mask] = priority_map["gray"]

        # Default: fill remaining non-black pixels
        default_mask = (cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) > 0) & (priority_layer == 9999)
        for c in range(3):
            overlay[:, :, c][default_mask] = frame[:, :, c][default_mask]
        priority_layer[default_mask] = 1000

    # Upscale if requested
    if target_size:
        overlay = cv2.resize(overlay, target_size, interpolation=cv2.INTER_LANCZOS4)
    elif scale_factor != 1:
        h, w = overlay.shape[:2]
        overlay = cv2.resize(
            overlay,
            (w * scale_factor, h * scale_factor),
            interpolation=cv2.INTER_LANCZOS4
        )

    return overlay


# Example usage
video_paths = [
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\v1_full_course\collision_scalar_success.mp4",
]
output_paths = [
    r"C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\v1_full_course\example_race.png",
]

for i, video_path in enumerate(video_paths):
    print(f"Processing: {video_path}")
    overlay = overlay_frames(video_path, color_priority=None, scale_factor=1)

    overlay_rgb = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(overlay_rgb)

    # Save
    img.save(output_paths[i], quality=100, subsampling=0)
    print(f"Grid with titles saved to {output_paths[i]}")

Processing: C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\v1_full_course\collision_scalar_success.mp4
Grid with titles saved to C:\Users\toazb\Documents\GitHub\race_simulation\videos\edge_cases\v1_full_course\example_race.png
